In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Project/2023/BIGCON/

/content/drive/MyDrive/Project/2023/BIGCON


In [3]:
import pandas as pd
import numpy as np

In [4]:
raw_data = pd.read_csv('2023빅콘테스트_빅데이터플랫폼활용분야_지정주제(부동산)_데이터셋/필지단위 소상공인 매출등급 정보.csv', encoding = 'cp949')
match_data = pd.read_csv('필지번호_행정동.csv', encoding = 'cp949')
test_data = pd.read_excel('2023빅콘테스트_빅데이터플랫폼활용분야_지정주제(부동산)_데이터셋/소상공인 매출등급 예상 대상 필지.xlsx')

In [5]:
# train이면 1, test면 0 변수 생성
raw_data['train'] = 1

In [6]:
# test 데이터는 202301과 202302가 열별로 되어있어 각각 데이터프레임 만든 후 raw데이터 밑으로 붙이기
test_2301 = test_data[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2302 = test_data[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2301['DATA_CRTR_YM'] = 202301
test_2302['DATA_CRTR_YM'] = 202302
test_2301['train'] = 0
test_2302['train'] = 0

<ipython-input-6-9ac68aabacc8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2301['DATA_CRTR_YM'] = 202301
<ipython-input-6-9ac68aabacc8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2302['DATA_CRTR_YM'] = 202302


In [7]:
test_2301 = test_2301[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'train']]
test_2302 = test_2302[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'train']]

In [8]:
raw_data = pd.concat([raw_data, test_2301, test_2302])

In [9]:
print(raw_data)
print(match_data.head())

      DATA_CRTR_YM            LT_UNQ_NO  STDG_EMD_CD INDUSTRY_CD  SLS_GRD  \
0           202201  1111010100100030100     11110101         A03      4.0   
1           202201  1111010100100480000     11110101         A01      2.0   
2           202201  1111010100100590001     11110101         A11      3.0   
3           202201  1111010100100590005     11110101         A03      2.0   
4           202201  1111010100100690000     11110101         A01      2.0   
...            ...                  ...          ...         ...      ...   
9995        202302  1130510100100830004     11305101         A03      NaN   
9996        202302  1135010300107450000     11350103         A13      NaN   
9997        202302  1150010300103400017     11500103         A06      NaN   
9998        202302  1144012700116070000     11440127         A03      NaN   
9999        202302  1126010400100770063     11260104         A03      NaN   

      train  
0         1  
1         1  
2         1  
3         1  
4    

In [10]:
# 신사동이 관악구, 강남구 둘 다에 있어서 처리해줘야함. 이외에 구는 경계에 걸친 애들
for i in range(len(match_data)):
    if match_data['ADM_NM'][i] == '신사동':
        if (match_data['SGG_NM'][i] == '관악구') or (match_data['SGG_NM'][i] == '동작구'):
            match_data['ADM_NM'][i] = '신사동1'
        elif (match_data['SGG_NM'][i] == '강남구') or (match_data['SGG_NM'][i] == '서초구') or (match_data['SGG_NM'][i] == '용산구')or (match_data['SGG_NM'][i] == '성동구'):
            match_data['ADM_NM'][i] = '신사동2'

<ipython-input-10-e2790476f055>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_data['ADM_NM'][i] = '신사동2'
<ipython-input-10-e2790476f055>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_data['ADM_NM'][i] = '신사동1'


raw데이터에서 매칭 안되는 필지번호는 분리

In [11]:
X = pd.DataFrame(raw_data['LT_UNQ_NO'].unique())
Y = pd.DataFrame(match_data['PNU'].unique())
PNU_NOTIN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique())-set(match_data['PNU'].unique()))
PNU_IN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique()) & set(match_data['PNU'].unique()))
PNU_NOTIN_data.rename(columns={0:'PNU'}, inplace=True)
PNU_IN_data.rename(columns={0:'PNU'}, inplace=True)

In [12]:
print(len(X))
print(len(PNU_IN_data) + len(PNU_NOTIN_data))
print(len(PNU_IN_data))
print(len(PNU_NOTIN_data))

74303
74303
74199
104


In [13]:
raw_in_match = raw_data[raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]
raw_notin_match = raw_data[~raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]

In [14]:
print(len(raw_data))
print(len(raw_in_match) + len(raw_notin_match))
print(len(raw_in_match))
print(len(raw_notin_match))

1176040
1176040
1172902
3138


In [15]:
# 데이터에 있는 필지번호와 match를 merge
df = pd.merge(PNU_IN_data, match_data, how='left')

# 중복 필지 중 하나만 남기고 제거
df.drop_duplicates(subset='PNU', keep='first', inplace=True, ignore_index=False)

In [16]:
print(len(PNU_IN_data))
print(len(df))

74199
74199


In [17]:
# raw데이터에 행정동명 합치기
raw_in_match = pd.merge(raw_in_match, df[['PNU', 'ADM_CD', 'ADM_NM']], how='left', left_on = 'LT_UNQ_NO', right_on = 'PNU')
raw_in_match = raw_in_match[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM', 'INDUSTRY_CD', 'SLS_GRD', 'train']]
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 1172901
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   train         1172902 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 80.5+ MB


### raw 데이터에 행정동별 노인인구수 붙이기

일단 만 65세 이상 기준으로 전체, 남, 여 각각 만들었음

In [18]:
# 행정동별 노인인구수
p2201 = pd.read_csv('행정동별_인구수/202201_202201_연령별인구현황_월간.csv', encoding = 'utf-8')
p2202 = pd.read_csv('행정동별_인구수/202202_202202_연령별인구현황_월간.csv', encoding = 'utf-8')
p2203 = pd.read_csv('행정동별_인구수/202203_202203_연령별인구현황_월간.csv', encoding = 'utf-8')
p2204 = pd.read_csv('행정동별_인구수/202204_202204_연령별인구현황_월간.csv', encoding = 'utf-8')
p2205 = pd.read_csv('행정동별_인구수/202205_202205_연령별인구현황_월간.csv', encoding = 'utf-8')
p2206 = pd.read_csv('행정동별_인구수/202206_202206_연령별인구현황_월간.csv', encoding = 'utf-8')
p2207 = pd.read_csv('행정동별_인구수/202207_202207_연령별인구현황_월간.csv', encoding = 'utf-8')
p2208 = pd.read_csv('행정동별_인구수/202208_202208_연령별인구현황_월간.csv', encoding = 'utf-8')
p2209 = pd.read_csv('행정동별_인구수/202209_202209_연령별인구현황_월간.csv', encoding = 'utf-8')
p2210 = pd.read_csv('행정동별_인구수/202210_202210_연령별인구현황_월간.csv', encoding = 'utf-8')
p2211 = pd.read_csv('행정동별_인구수/202211_202211_연령별인구현황_월간.csv', encoding = 'utf-8')
p2212 = pd.read_csv('행정동별_인구수/202212_202212_연령별인구현황_월간.csv', encoding = 'utf-8')
p2301 = pd.read_csv('행정동별_인구수/202301_202301_연령별인구현황_월간.csv', encoding = 'utf-8')
p2302 = pd.read_csv('행정동별_인구수/202302_202302_연령별인구현황_월간.csv', encoding = 'utf-8')

In [19]:
# 신사동1, 신사동2 구분
data_list = [p2201, p2202, p2203, p2204, p2205, p2206, p2207, p2208, p2209, p2210, p2211, p2212, p2301, p2302]

for i in data_list :
  i.iloc[332, 1] = '관악구 신사동1'
  i.iloc[358, 1] = '강남구 신사동2'

In [20]:
# 행정동별 노인인구수에서 행정구역을 이용해서 행정동 만들기
p2201['ADM_NM'] = p2201['행정기관'].apply(lambda x: x.split()[1])
p2202['ADM_NM'] = p2202['행정기관'].apply(lambda x: x.split()[1])
p2203['ADM_NM'] = p2203['행정기관'].apply(lambda x: x.split()[1])
p2204['ADM_NM'] = p2204['행정기관'].apply(lambda x: x.split()[1])
p2205['ADM_NM'] = p2205['행정기관'].apply(lambda x: x.split()[1])
p2206['ADM_NM'] = p2206['행정기관'].apply(lambda x: x.split()[1])
p2207['ADM_NM'] = p2207['행정기관'].apply(lambda x: x.split()[1])
p2208['ADM_NM'] = p2208['행정기관'].apply(lambda x: x.split()[1])
p2209['ADM_NM'] = p2209['행정기관'].apply(lambda x: x.split()[1])
p2210['ADM_NM'] = p2210['행정기관'].apply(lambda x: x.split()[1])
p2211['ADM_NM'] = p2211['행정기관'].apply(lambda x: x.split()[1])
p2212['ADM_NM'] = p2212['행정기관'].apply(lambda x: x.split()[1])
p2301['ADM_NM'] = p2301['행정기관'].apply(lambda x: x.split()[1])
p2302['ADM_NM'] = p2302['행정기관'].apply(lambda x: x.split()[1])

In [21]:
p2201.head()

,행정기관코드,행정기관,65세이상,65세이상_남,65세이상_여,ADM_NM
0,1111051500,종로구 청운효자동,"2,144",875,"1,269",청운효자동
1,1111053000,종로구 사직동,"1,759",767,992,사직동
2,1111054000,종로구 삼청동,605,259,346,삼청동
3,1111055000,종로구 부암동,"1,799",795,"1,004",부암동
4,1111056000,종로구 평창동,"3,421","1,490","1,931",평창동


In [22]:
# 월별로 merge
raw_2201 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202201], p2201[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2202 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202202], p2202[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2203 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202203], p2203[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2204 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202204], p2204[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2205 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202205], p2205[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2206 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202206], p2206[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2207 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202207], p2207[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2208 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202208], p2208[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2209 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202209], p2209[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2210 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202210], p2210[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2211 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202211], p2211[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2212 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202212], p2212[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2301 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202301], p2301[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2302 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202302], p2302[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')

In [23]:
# 월별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_2201, raw_2202, raw_2203, raw_2204, raw_2205, raw_2206, raw_2207, raw_2208, raw_2209,
                          raw_2210, raw_2211, raw_2212, raw_2301, raw_2302])
raw_in_match.reset_index(drop = True, inplace = True)

In [24]:
raw_in_match['65세이상'] = raw_in_match['65세이상'].apply(lambda x: int(x.replace(',','')))
raw_in_match['65세이상_남'] = raw_in_match['65세이상_남'].apply(lambda x: int(x.replace(',','')))
raw_in_match['65세이상_여'] = raw_in_match['65세이상_여'].apply(lambda x: int(x.replace(',','')))

In [25]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172902 entries, 0 to 1172901
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   train         1172902 non-null  int64  
 8   65세이상         1172902 non-null  int64  
 9   65세이상_남       1172902 non-null  int64  
 10  65세이상_여       1172902 non-null  int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 98.4+ MB


In [26]:
raw_in_match

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,ADM_CD,ADM_NM,INDUSTRY_CD,SLS_GRD,train,65세이상,65세이상_남,65세이상_여
0,202201,1111010100100030100,11110101,11010720,청운효자동,A03,4.0,1,2144,875,1269
1,202201,1111010100100480000,11110101,11010720,청운효자동,A01,2.0,1,2144,875,1269
2,202201,1111010100100590001,11110101,11010720,청운효자동,A11,3.0,1,2144,875,1269
3,202201,1111010100100590005,11110101,11010720,청운효자동,A03,2.0,1,2144,875,1269
4,202201,1111010100100690000,11110101,11010720,청운효자동,A01,2.0,1,2144,875,1269
...,...,...,...,...,...,...,...,...,...,...,...
1172897,202302,1130510100100830004,11305101,11090710,송중동,A03,NaN,0,6234,2748,3486
1172898,202302,1135010300107450000,11350103,11110560,공릉2동,A13,NaN,0,5965,2691,3274
1172899,202302,1150010300103400017,11500103,11160630,화곡8동,A06,NaN,0,4596,2007,2589
1172900,202302,1144012700116070000,11440127,11140740,상암동,A03,NaN,0,4279,1801,2478


### raw_notin_match 합치기

필지번호 앞 10자리 기준으로 평균

In [27]:
pnu_match = match_data['PNU'].astype(str)
pnu_notin_match = raw_notin_match['LT_UNQ_NO'].astype(str)

def str_slicing(s):
    return s[:10]

pnu_match_slice = list(map(str_slicing, pnu_match))
pnu_notin_match_slice = list(map(str_slicing, pnu_notin_match))

In [28]:
set(pnu_notin_match_slice) - set(pnu_match_slice) # 10번째 자리까지 slice하면 match에 있는 필지번호(10자리까지)에 모두 포함됨

set()

raw_notin_match에 slicing한 필지번호 추가해서 matching

In [29]:
raw_notin_match['LT_UNQ_NO_slice'] = raw_notin_match['LT_UNQ_NO'].astype(str)
raw_notin_match['LT_UNQ_NO_slice'] = list(map(str_slicing, raw_notin_match['LT_UNQ_NO_slice']))
match_data['PNU_slice'] = match_data['PNU'].astype(str)
match_data['PNU_slice'] = list(map(str_slicing, match_data['PNU_slice']))

<ipython-input-29-55731f807e2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_notin_match['LT_UNQ_NO_slice'] = raw_notin_match['LT_UNQ_NO'].astype(str)
<ipython-input-29-55731f807e2f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_notin_match['LT_UNQ_NO_slice'] = list(map(str_slicing, raw_notin_match['LT_UNQ_NO_slice']))


In [30]:
raw_notin_match

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,LT_UNQ_NO_slice
280,202201,1111011000102450006,11110110,A03,2.0,1,1111011000
12662,202201,1120010700103770000,11200107,A01,3.0,1,1120010700
12663,202201,1120010700103770000,11200107,A08,2.0,1,1120010700
17432,202201,1121510500108610000,11215105,A08,4.0,1,1121510500
24678,202201,1126010500106150005,11260105,A13,5.0,1,1126010500
...,...,...,...,...,...,...,...
6356,202302,1165010100104500013,11650101,A01,NaN,0,1165010100
7203,202302,1165010100110180001,11650101,A01,NaN,0,1165010100
7378,202302,1138011400100250000,11380114,A06,NaN,0,1138011400
7384,202302,1171010800106520000,11710108,A01,NaN,0,1171010800


In [31]:
raw_notin_match1 = raw_notin_match

In [32]:
match_data.drop(columns='PNU', inplace=True)
match_data.drop_duplicates(inplace=True)

In [33]:
match_data

,SGG_CD,SGG_NM,EMD_CD,EMD_NM,ADM_CD,ADM_NM,PNU_slice
0,11110,종로구,11110101,청운동,11010720,청운효자동,1111010100
6,11110,종로구,11110101,청운동,11010550,부암동,1111010100
792,11110,종로구,11110101,청운동,11010540,삼청동,1111010100
825,11110,종로구,11110102,신교동,11010720,청운효자동,1111010200
1220,11110,종로구,11110103,궁정동,11010720,청운효자동,1111010300
...,...,...,...,...,...,...,...
937427,11740,강동구,11740109,천호동,11240510,풍납1동,1174010900
938127,11740,강동구,11740109,천호동,11050630,광장동,1174010900
938483,11740,강동구,11740110,강일동,11250750,강일동,1174011000
939255,11740,강동구,11740110,강일동,11250560,고덕2동,1174011000


In [34]:
raw_notin_match = pd.merge(raw_notin_match, match_data[['PNU_slice', 'ADM_CD', 'ADM_NM']], left_on='LT_UNQ_NO_slice', right_on='PNU_slice', how = 'left')
raw_notin_match.drop(columns=['PNU_slice'], axis=1, inplace=True)

In [35]:
raw_notin_match = raw_notin_match[['DATA_CRTR_YM', 'LT_UNQ_NO', 'LT_UNQ_NO_slice', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM', 'INDUSTRY_CD', 'SLS_GRD', 'train']]

In [36]:
raw_notin_match

,DATA_CRTR_YM,LT_UNQ_NO,LT_UNQ_NO_slice,STDG_EMD_CD,ADM_CD,ADM_NM,INDUSTRY_CD,SLS_GRD,train
0,202201,1111011000102450006,1111011000,11110110,11010720,청운효자동,A03,2.0,1
1,202201,1111011000102450006,1111011000,11110110,11010530,사직동,A03,2.0,1
2,202201,1120010700103770000,1120010700,11200107,11040550,사근동,A01,3.0,1
3,202201,1120010700103770000,1120010700,11200107,11040540,마장동,A01,3.0,1
4,202201,1120010700103770000,1120010700,11200107,11040560,행당1동,A01,3.0,1
...,...,...,...,...,...,...,...,...,...
28329,202302,1171010800106400007,1171010800,11710108,11240530,거여1동,A11,NaN,0
28330,202302,1171010800106400007,1171010800,11710108,11240650,가락본동,A11,NaN,0
28331,202302,1171010800106400007,1171010800,11710108,11240690,문정2동,A11,NaN,0
28332,202302,1171010800106400007,1171010800,11710108,11240660,가락1동,A11,NaN,0


In [37]:
pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202201], p2201[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')

,DATA_CRTR_YM,LT_UNQ_NO,LT_UNQ_NO_slice,STDG_EMD_CD,ADM_CD,ADM_NM,INDUSTRY_CD,SLS_GRD,train,65세이상,65세이상_남,65세이상_여
0,202201,1111011000102450006,1111011000,11110110,11010720,청운효자동,A03,2.0,1,"2,144",875,"1,269"
1,202201,1111011000102450006,1111011000,11110110,11010530,사직동,A03,2.0,1,"1,759",767,992
2,202201,1120010700103770000,1120010700,11200107,11040550,사근동,A01,3.0,1,"1,509",642,867
3,202201,1120010700103770000,1120010700,11200107,11040540,마장동,A01,3.0,1,"3,944","1,748","2,196"
4,202201,1120010700103770000,1120010700,11200107,11040560,행당1동,A01,3.0,1,"2,143",967,"1,176"
...,...,...,...,...,...,...,...,...,...,...,...,...
2147,202201,1174010300105260000,1174010300,11740103,11250750,강일동,A03,5.0,1,"5,363","2,327","3,036"
2148,202201,1174010300105260000,1174010300,11740103,11250760,상일1동,A03,5.0,1,"4,430","2,138","2,292"
2149,202201,1174010300105260000,1174010300,11740103,11250560,고덕2동,A03,5.0,1,"3,022","1,460","1,562"
2150,202201,1174010300105260000,1174010300,11740103,11250550,고덕1동,A03,5.0,1,"2,871","1,318","1,553"


In [38]:
# 월별로 merge
raw_2201 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202201], p2201[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2202 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202202], p2202[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2203 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202203], p2203[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2204 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202204], p2204[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2205 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202205], p2205[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2206 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202206], p2206[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2207 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202207], p2207[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2208 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202208], p2208[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2209 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202209], p2209[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2210 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202210], p2210[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2211 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202211], p2211[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2212 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202212], p2212[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2301 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202301], p2301[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2302 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] == 202302], p2302[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')

In [39]:
# 월별 데이터 하나로 합치기
raw_notin_match = pd.concat([raw_2201, raw_2202, raw_2203, raw_2204, raw_2205, raw_2206, raw_2207, raw_2208, raw_2209,
                             raw_2210, raw_2211, raw_2212, raw_2301, raw_2302])
raw_notin_match.reset_index(drop = True, inplace = True)

In [40]:
raw_notin_match

,DATA_CRTR_YM,LT_UNQ_NO,LT_UNQ_NO_slice,STDG_EMD_CD,ADM_CD,ADM_NM,INDUSTRY_CD,SLS_GRD,train,65세이상,65세이상_남,65세이상_여
0,202201,1111011000102450006,1111011000,11110110,11010720,청운효자동,A03,2.0,1,"2,144",875,"1,269"
1,202201,1111011000102450006,1111011000,11110110,11010530,사직동,A03,2.0,1,"1,759",767,992
2,202201,1120010700103770000,1120010700,11200107,11040550,사근동,A01,3.0,1,"1,509",642,867
3,202201,1120010700103770000,1120010700,11200107,11040540,마장동,A01,3.0,1,"3,944","1,748","2,196"
4,202201,1120010700103770000,1120010700,11200107,11040560,행당1동,A01,3.0,1,"2,143",967,"1,176"
...,...,...,...,...,...,...,...,...,...,...,...,...
28329,202302,1171010800106400007,1171010800,11710108,11240530,거여1동,A11,NaN,0,"2,524","1,170","1,354"
28330,202302,1171010800106400007,1171010800,11710108,11240650,가락본동,A11,NaN,0,"4,068","1,980","2,088"
28331,202302,1171010800106400007,1171010800,11710108,11240690,문정2동,A11,NaN,0,"4,227","2,054","2,173"
28332,202302,1171010800106400007,1171010800,11710108,11240660,가락1동,A11,NaN,0,"3,846","1,867","1,979"


In [41]:
raw_notin_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28334 entries, 0 to 28333
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DATA_CRTR_YM     28334 non-null  int64  
 1   LT_UNQ_NO        28334 non-null  int64  
 2   LT_UNQ_NO_slice  28334 non-null  object 
 3   STDG_EMD_CD      28334 non-null  int64  
 4   ADM_CD           28334 non-null  int64  
 5   ADM_NM           28334 non-null  object 
 6   INDUSTRY_CD      28334 non-null  object 
 7   SLS_GRD          28084 non-null  float64
 8   train            28334 non-null  int64  
 9   65세이상            28334 non-null  object 
 10  65세이상_남          28334 non-null  object 
 11  65세이상_여          28334 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.6+ MB


In [42]:
raw_notin_match['65세이상'] = raw_notin_match['65세이상'].apply(lambda x: int(x.replace(',','')))
raw_notin_match['65세이상_남'] = raw_notin_match['65세이상_남'].apply(lambda x: int(x.replace(',','')))
raw_notin_match['65세이상_여'] = raw_notin_match['65세이상_여'].apply(lambda x: int(x.replace(',','')))

In [43]:
raw_notin_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28334 entries, 0 to 28333
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DATA_CRTR_YM     28334 non-null  int64  
 1   LT_UNQ_NO        28334 non-null  int64  
 2   LT_UNQ_NO_slice  28334 non-null  object 
 3   STDG_EMD_CD      28334 non-null  int64  
 4   ADM_CD           28334 non-null  int64  
 5   ADM_NM           28334 non-null  object 
 6   INDUSTRY_CD      28334 non-null  object 
 7   SLS_GRD          28084 non-null  float64
 8   train            28334 non-null  int64  
 9   65세이상            28334 non-null  int64  
 10  65세이상_남          28334 non-null  int64  
 11  65세이상_여          28334 non-null  int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.6+ MB


In [44]:
# 월(DATA_CRTR_YM), 필지번호(LT_UNQ_NO), 업종코드(INDUSTRY_CD) 기준으로 평균내기
grouped = raw_notin_match.groupby(['DATA_CRTR_YM', 'LT_UNQ_NO', 'INDUSTRY_CD'])[['65세이상', '65세이상_남', '65세이상_여']].mean().reset_index()

In [45]:
grouped

,DATA_CRTR_YM,LT_UNQ_NO,INDUSTRY_CD,65세이상,65세이상_남,65세이상_여
0,202201,1111011000102450006,A03,1951.500000,821.000,1130.500000
1,202201,1120010700103770000,A01,2773.222222,1199.000,1574.222222
2,202201,1120010700103770000,A08,2773.222222,1199.000,1574.222222
3,202201,1121510500108610000,A08,3537.076923,1610.000,1927.076923
4,202201,1126010500106150005,A13,5529.800000,2470.800,3059.000000
...,...,...,...,...,...,...
3133,202302,1171010800106510000,A10,3830.375000,1799.375,2031.000000
3134,202302,1171010800106510014,A11,3830.375000,1799.375,2031.000000
3135,202302,1171010800106510014,A13,3830.375000,1799.375,2031.000000
3136,202302,1171010800106520000,A01,3830.375000,1799.375,2031.000000


In [46]:
raw_notin_match2 = pd.merge(raw_notin_match1, grouped, on=['DATA_CRTR_YM', 'LT_UNQ_NO', 'INDUSTRY_CD'])
raw_notin_match2.drop(columns='LT_UNQ_NO_slice', inplace=True)

In [47]:
raw_notin_match2

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,65세이상,65세이상_남,65세이상_여
0,202201,1111011000102450006,11110110,A03,2.0,1,1951.500000,821.000000,1130.500000
1,202201,1120010700103770000,11200107,A01,3.0,1,2773.222222,1199.000000,1574.222222
2,202201,1120010700103770000,11200107,A08,2.0,1,2773.222222,1199.000000,1574.222222
3,202201,1121510500108610000,11215105,A08,4.0,1,3537.076923,1610.000000,1927.076923
4,202201,1126010500106150005,11260105,A13,5.0,1,5529.800000,2470.800000,3059.000000
...,...,...,...,...,...,...,...,...,...
3133,202302,1165010100104500013,11650101,A01,NaN,0,3618.307692,1624.076923,1994.230769
3134,202302,1165010100110180001,11650101,A01,NaN,0,3618.307692,1624.076923,1994.230769
3135,202302,1138011400100250000,11380114,A06,NaN,0,5946.600000,2573.600000,3373.000000
3136,202302,1171010800106520000,11710108,A01,NaN,0,3830.375000,1799.375000,2031.000000


In [48]:
raw_in_match.columns

Index(['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM',
       'INDUSTRY_CD', 'SLS_GRD', 'train', '65세이상', '65세이상_남', '65세이상_여'],
      dtype='object')

In [49]:
raw_notin_match2.columns

Index(['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'SLS_GRD',
       'train', '65세이상', '65세이상_남', '65세이상_여'],
      dtype='object')

In [50]:
# raw_notin_match2에는 행정동코드, 행정동명, 생활밀접업종 열이 없으므로 raw_in_match에서도 drop
raw_in_match.drop(columns=['ADM_CD', 'ADM_NM'], inplace=True)

In [51]:
raw_data1 = pd.concat([raw_in_match, raw_notin_match2]).sort_values(by='DATA_CRTR_YM').reset_index(drop=True)
raw_data1.info() # 초기 raw_data의 length인 1176040과 일치

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176040 entries, 0 to 1176039
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1176040 non-null  int64  
 1   LT_UNQ_NO     1176040 non-null  int64  
 2   STDG_EMD_CD   1176040 non-null  int64  
 3   INDUSTRY_CD   1176040 non-null  object 
 4   SLS_GRD       1156040 non-null  float64
 5   train         1176040 non-null  int64  
 6   65세이상         1176040 non-null  float64
 7   65세이상_남       1176040 non-null  float64
 8   65세이상_여       1176040 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 80.8+ MB


In [52]:
# 이거 합치면 됨
raw_data1

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,65세이상,65세이상_남,65세이상_여
0,202201,1111010100100030100,11110101,A03,4.0,1,2144.000,875.000,1269.0
1,202201,1156011100104460000,11560111,A01,5.0,1,3386.000,1506.000,1880.0
2,202201,1156011100104450001,11560111,A03,3.0,1,3386.000,1506.000,1880.0
3,202201,1156011100104390000,11560111,A04,4.0,1,3386.000,1506.000,1880.0
4,202201,1156011100104340000,11560111,A01,5.0,1,3386.000,1506.000,1880.0
...,...,...,...,...,...,...,...,...,...
1176035,202302,1147010200104060293,11470102,A01,NaN,0,3537.000,1632.000,1905.0
1176036,202302,1159010100100950000,11590101,A01,NaN,0,6007.000,2582.000,3425.0
1176037,202302,1121510100102110002,11215101,A06,NaN,0,2957.000,1310.000,1647.0
1176038,202302,1174010200101940007,11740102,A03,NaN,0,3189.000,1552.000,1637.0


In [53]:
raw_data1.to_csv('행정동별_노인인구수.csv', index = False, encoding = 'cp949')